## Basic

In [2]:
import os
import cv2
import hashlib
from collections import defaultdict

In [3]:
# Function to calculate the hash of an image
def dhash(image, hash_size=8):
    resized = cv2.resize(image, (hash_size + 1, hash_size))
    diff = resized[:, 1:] > resized[:, :-1]
    return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

In [4]:
# Define the input folder containing the images
input_folder = 'pics'
output_folder = 'isolated items'

## Unique Object Detection

In [10]:
yolo_net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in yolo_net.getUnconnectedOutLayers()]

error: OpenCV(4.8.0) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/dnn/src/darknet/darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov3.cfg in function 'readNetFromDarknet'


In [9]:
def detect_objects(image):
    height, width, channels = image.shape
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)
    outs = yolo_net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Adjust confidence threshold as needed
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    return class_ids, confidences, boxes

In [5]:
# Dictionary to store unique images based on their hash
unique_images = defaultdict(list)

# Iterate through the images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        img_path = os.path.join(input_folder, filename)
        image = cv2.imread(img_path)
        class_ids, _, _ = detect_objects(image)

        # Convert class IDs to a hash to represent unique objects in the image
        obj_hash = hashlib.md5(str(class_ids).encode()).hexdigest()

        # Add the image to the dictionary with its hash as the key
        unique_objects[obj_hash].append(img_path)

In [6]:
# Iterate through the dictionary and save unique images
for obj_hash, image_paths in unique_objects.items():
    if len(image_paths) == 1:
        img_path = image_paths[0]
        filename = os.path.basename(img_path)
        output_path = os.path.join(output_folder, filename)
        os.rename(img_path, output_path)
        print(f"Unique object image saved: {output_path}")

Unique image saved: isolated items/food.png
